# Final Project for CS 6.338
# Emily Crabb

# Parallel molecular dyanmics simulation implementation in Julia

some sort of description of project !!!!!!!!

Sources:

https://people.sc.fsu.edu/~jburkardt/py_src/md/md.py

https://www.saylor.org/site/wp-content/uploads/2011/06/MA221-6.1.pdf


To do (in no particular order):

Allow systems of finite size (both periodic and non-periodic)

Convert to script

Allow user to specify values of parameters

Read initial conditions from file

Save data to file

Display output

More complicated particle interactions


In [8]:
# Generate strengths of interactions between particle types

function gen_interaction(num_part_types)
    interaction_params = zeros(Float64, num_part_types,num_part_types)
    for i=1:num_part_types
        for j = 1:num_part_types
            if (i==j) # Self-interaction is randomly repulsive
                interaction_params[i,j] = -1*rand(Float64)
            elseif (i<j) # Others randomly attractive
                val = rand(Float64)
                interaction_params[i,j] = val
                interaction_params[j,i] = val
            end
        end        
    end
    
    return interaction_params
end

gen_interaction (generic function with 1 method)

In [15]:
# Global constants 
# If convert to script, some should be user inputs

const number_of_steps = 500; # Number of steps to execute in simulation
const dim = 3; # Dimensions of simulation
const box_size = 10; # Size of one side of box
#const periodic = false; # Whether simulation is periodic
const part_num = 500; # Number of particles in simulation
const dt = 0.1; # Time step
const num_part_types = 2; # Number of types of particles
const interaction_params = gen_interaction(num_part_types); # Interations parameters for types of particles
const mass_parts = rand(Float64, num_part_types); # Masses of types of particles

In [16]:
# Currently start with zero velocity and acceleration
# Currently start with random positions and randomly assigned particle types

function initialize(part_num, dim, box_size, num_part_types)
    
    pos = box_size*rand(Float64, part_num, dim) # Initialized to be randomly placed within a box
    
    vel = zeros(Float64, part_num, dim) # Initialized to zero
    acc = zeros(Float64, part_num, dim) # Initialized to zero
    
    part_types = rand(1:num_part_types, part_num)
        
    return pos, vel, acc, part_types
end

initialize (generic function with 1 method)

In [33]:
# Velocity Verlet Algorithm
# Currently system size is infinite (positions not constrained)

function step_update(part_num, dim, pos, vel, acc, force, part_types, mass_parts, dt)
    
    for i = 1:part_num # For every particle
        mass = mass_parts[part_types[i]]
        for j = 1:dim # For each dimension
            pos[i,j] = pos[i,j] + vel[i,j]*dt + 0.5*acc[i,j]*dt^2 # x(t+Δt) = x(t) + v(t)Δt + 1/2*a(t)(Δt)^2
            vel[i,j] = vel[i,j] + 0.5*(acc[i,j] + force[i,j]/mass)*dt # v(t+Δt) = v(t) + 1/2*(a(t)+a(t+Δt))Δt
            acc[i,j] = force[i,j]/mass # a = F/m
        end
    end
    
    return pos, vel, acc
end

step_update (generic function with 2 methods)

In [48]:
# 1/r^2 interactions currently

function find_force(part_num, dim, pos, vel, acc, part_types, interaction_params, mass_parts)
    
    force = zeros(part_num, dim)
    
    for i = 1:part_num # For every particle
        mass = mass_parts[part_types[i]]
        for k = 1:part_num # Contribution from every other particle
            if (i != k) # No self-interaction
                for j = 1:dim # For each dimension
                    int_strength = interaction_params[part_types[i],part_types[k]] # Strength of interaction between particles
                    force[i,j] = int_strength / (pos[i,j] - pos[k,j])^2 # 1/r^2 interaction
                end
            end
        end
    end
    
    return force
end

find_force (generic function with 2 methods)

In [49]:
# Main - where program executes
# Convert to main(args) if convert to script

step = 0

pos, vel, acc, part_types = initialize(part_num, dim, box_size, num_part_types)
force = find_force(part_num, dim, pos, vel, acc, part_types, interaction_params, mass_parts)
#println(force)
pos_init = pos

for i = 1:number_of_steps
    pos, vel, acc = step_update(part_num, dim, pos, vel, acc, force, part_types, mass_parts, dt) 
    force = find_force(part_num, dim, pos, vel, acc, part_types, interaction_params, mass_parts)
end


[0.838386 0.0879675 0.0952708; 0.571179 0.0124249 0.0402006; -0.00996722 -0.0224354 -0.0372346; 0.0313876 0.0466576 0.104624; -0.0134648 -1.38786 -75.4116; -10.0782 -0.0170058 -322.071; 0.0573071 0.0126615 0.212146; -0.0156515 -0.0119885 -0.0498381; -0.0237474 -0.0183134 -0.107587; -0.0148975 -0.590432 -0.0413047; -0.280674 -0.0829685 -0.844564; -0.023556 -0.019328 -0.0918716; -0.0103531 -0.0542084 -0.104273; -0.113407 -0.0105801 -0.0335397; -0.715383 -0.367858 -0.0450668; -0.798475 -0.0382838 -0.0598508; 0.121271 1.53316 0.0744561; -0.0227853 -1.64972 -0.115141; -0.0135982 -0.160983 -0.0574874; 0.0220386 0.0163907 0.0357044; -0.131968 -0.252774 -0.028671; 0.0100273 0.0375498 2.16105; -2.46684 -0.0312286 -113.029; 0.00991847 0.00781163 0.0231125; -0.120042 -0.234825 -0.0233497; -0.0138454 -0.0152193 -0.0367949; 0.00859311 0.0112319 0.0347722; -0.0361375 -0.635581 -0.058975; 0.0261744 0.00978429 0.197331; -0.0235544 -0.0406407 -0.0501675; 0.00706634 1.57739 0.0206339; 0.121134 0.0126016

; -0.256833 -0.0111765 -0.188682; 0.00686957 0.131375 0.0309085; 0.00695313 0.0229582 0.0669929; -0.0399961 -0.421875 -0.187755; 0.158051 0.0213555 0.0140911; -0.0870172 -0.0150383 -0.161304; -0.0198307 -0.0427858 -0.0561041; 0.00709784 12.7471 0.0349094; -1.65281 -0.0554349 -0.285894; 0.226256 0.00941231 0.017181; 0.0351845 0.0192912 0.227784; 0.138534 0.00987534 0.158574; 0.198264 0.0111528 0.0653333; 0.140571 12.9429 0.018749; -0.154546 -0.173662 -0.0417995; -0.0170732 -0.0947912 -0.382346; 0.0306916 0.15832 0.426529; -5.85707 -0.018899 -1.39615; -0.0189528 -0.213336 -0.0876497; -1.26115 -0.129851 -0.0609505; 0.00913754 4.56185 0.573942; 0.141822 0.0129177 0.0168697; -0.0126493 -0.0149619 -0.0647584; -0.0182516 -0.0169278 -0.0218438; -0.0113109 -0.0384561 -0.216974; 1.03728 0.0876702 0.0524737; -0.314066 -0.0962005 -0.239401; 2.27509 0.023527 0.251286; -0.0144525 -0.0105623 -0.90036; -0.00959677 -0.0117815 -0.0485581; 0.0816805 0.0146969 0.19434; 0.0092975 0.227371 4.10428; -0.41971